### Vorgehen:
1. beschränktere Parameter herausfinden durch Experimente mit num_exp=1 und reduce_dataset=0.2.
    Parameter: Backbone, kd(schhonmal testen)
2. Wie 1. aber mit num_exp=3.
    Parameter: classification, selection_method
- Dadurch schonmal grobe Infos sammeln und dann später stichprobenartig verifizieren

3. Mit den gesammelten Infos die Benchmarks übertreffen oder Schonmal Q2 und Q3 anfangen

## ---

### Sweep: classification_and_selection
- reduce_dataset=1.1 statt 0.2:   
  
|        | cifar100 | imagenetr | cub | dil_imagenetr | imageneta | vtab | cars | omnibenchmark | limited_domainnet |
|--------|----------|-----------|-----|--------------|-----------|------|------|--------------|-------------------|
| Selection| bayesian | bayesian | bayesian | bayesian | bayesian | bayesian | --- | bayesian | bayesian |
| Classification  | inv_ws/inv_kl | ws | inv_eucld | ws | ws | kl/eucld | --- | ws/kl/eucld | inv_ws/(kl) |  

- Alt: manche lr/ft_epochs waren immer noch schlecht -> sc_q1_some_datasets_classification_and_selection_get_better_param2 (me5norfz)
- Alt: generelle Tests: fj01kgs5, (new 14cyr3rb)
- Alt  mit opt lr und ft-epochhs lim_domainnet: sr9avnjp

## ---

### Sweep: backbone_and_kd
- Backbone: nbibpxto, KD: pj12t7ux

|               | cifar100 | imagenetr | cub | dil_imagenetr | imageneta | vtab | cars | omnibenchmark | limited_domainnet |
|---------------|----------|-----------|-----|--------------|-----------|------|------|--------------|-------------------|
| Backbone | egal | in21k | in21k | in21k | (in21k) | normal | --- | normal | normal |  
| KD | gleich | gleich | gleich | gleich | gleich | gleich | --- | gleich | <1% |

#### KD:
Bei cifar100: pj12t7ux nicht klar entscheidbar

- Alt: Bessere Parameter herausfinden damit nicht jede acc. 0.1 ist (tp49kob2)
- Alt: Cifar: Bei KD alpha hoch machen, da es bis jetzt immer egal ist: (pj12t7ux). Achtung: wenn kd=False ist kd-alpha egal weil es nicht benutzt wird. Habe ich KD richtig implementiert? Evtl aber auch ungünstige andere parameter gewählt, weil oft acc=random

## ---

### Sweep: lr_ftepoch
  
|        | cifar100 | imagenetr | cub | dil_imagenetr | imageneta | vtab | cars | omnibenchmark | limited_domainnet |
|--------|----------|-----------|-----|--------------|-----------|------|------|--------------|-------------------|
| Selection| bayesian | bayesian | bayesian | bayesian | bayesian | bayesian | --- | bayesian | bayesian |
| Classification  | inv_ws/inv_kl | ws | inv_eucld | ws | ws | kl/eucld | --- | ws/kl/eucld | inv_ws/(kl) |  

- Neu:wie siht der datensatz (domaine oder klassen), wie hsehen die distr. aus. größe des datensatzes und anzahl der images pro klasse. um im vorhinein schon classification auszuwählen. Layup als inspiration. Lr und ft epochs (sollte fest sein) gleich für backbones evtl auch für jeden datensatz (testen mit cifar und dann vtab (klein) und testen ob die lr wirklich so unterschidlich sind)x6p5pa6a und 2ejr8z6d

### Sweep qxb0hd5i: sweep_config_batchsize_optimization
T=10; moe_max_experts=5; reduce_dataset=1
|        | cifar100 | imagenetr | cub | dil_imagenetr | imageneta | vtab | cars | omnibenchmark | limited_domainnet | cddb |
|--------|----------|-----------|-----|--------------|-----------|------|------|--------------|-------------------| ---- |
| GPU Memory| 7851.74 | 9655.29 | 7228.88 | 9655.29 | 6656.36 | 7486.83 | 7186.94 | 9xxx.xx | --- | --- |


## ---

## Kyra Besprechung:
### 25.03.
1. Warum ist das Lernen so ungleich verteilt? Was ist das Kriterium für den Vergleich? Welche beiden Werte werden miteinander verglichen und wie kommen sie zu stande?
2. Acc.=0 ist Logikfehler. Was wird genau predicted und wie? Wie funktioniert der Prozess? Was ist der output und welche experten werden dabei wie benutzt?
3. Wie funktioniert das vergessen und wieviel wird vergessen? Wenn immer nur ein Experte lernt, vergisst er dann wie zu erwarten oder stärkter?
4. Random Selection als Vergleich und als Basis. Ich weis da ja definitiv wie der selection Prozess funktioniert, funktioniert denn alles andere wie zu erwarten?
5. Wenn die Performance random aussieht wird dann immer die gleiche Klasse predicted oder falsche unterschieldiche?  

Generelle Ideen:  
- predict_class_bayes vereinfachen und mich nicht zu doll an die SEED-Implementation halten. Einfach eine Liste mit allen Distr. und keine Aufteilung nach Experten. Würde den dim=3 Tensor vereinfachen. Ich muss für den seection Prozess aber Überblick behalten welcher Expert welche Klasse/Distr. gelernt hat.
- Man kann evtl. nicht einfach die features des ViTs nehmen sondern nur den ersten (cls?) Token nehmen?
- Pseudocode der Funktionen schreiben!! Ist sehr gut für verständniss und späteres beschreiben des Algs.
- Werden die distrs. richtig erstellt? was ist mit der symetrie/gespiegelten features in der distr.? whabe ich das noch drin? Da könnte der Fehler sein.

#### Selection (Warum wird das wissen so ungleich verteilt?):
4. 7s41oxdx angucken und random mit kl_div vergleichen. Wo liegt der fehler? kann ich durch den Vergleich fehlerquellen ausschließen?  
--> kurz angeguckt: kl_div ist besser als random, aber ich weiß nicht genau wann oder wieso
- kl div: https://wandb.ai/belaschindler-university-hamburg/Test%20project/runs/u89ipbvk/overview
- inverted kl div: https://wandb.ai/belaschindler-university-hamburg/Test%20project/runs/bcrcujyc
- evtl unnötig, wenn ich die versuche eh unten abgedeckt habe

#### Classification:





#### klassification aber auch selection (klassification in den Logs):  
  moe_max_expert:  
    values: [5]  
  selection_method:  
    values: ["kl_div", "inv_kl_div"]  
  finetune_epochs:  
    values: [10]  
  lr:  
    values: [0.005]  
  kl_div_test:  
    values: [0, 1, 2]  
    
Sweep: qubquo0z (alt, wahrscheinlich unnütz), a8ap2jey  

| Acc. | Mean | Min | Max |
|--------|----------|-----------|-----|
| kl_div| - | + | + |
| inv_kl_div | - | + | - |

+: > 0.2, -: < 0.1

Acc. Graphen sind alle sehr änlich: Erstmal gut (T<=5) und dann schmiert's ab

| Finetunung (Auswahl) | Mean | Min | Max |
|--------|----------|-----------|-----|
| kl_div| 2:1, 4:4 | 0:2, 2:2, 3:1 | 2:5 |
| inv_kl_div | 0:4, 3:1 | 4:1, 2:2, 3:2 | 0:2, 1:1, 3:1, 4:1 |

Beste Aufteilung: Maximum mit inverse_kl_div: Ich nehme die maximalen Abstände und dann den experten mit dem kleinsten maximalen Abstand  
Ist auch sonst gute selection für bessere Aufteilun. Weitere Bestätigung: sweep oxtwsjhw (Viele CUDA Errors)
Es korreliert aber nicht mit Acc.


| Klassifikation (Auswahl) | Mean | Min | Max |
|--------|----------|-----------|-----|
| kl_div| Richige Experten | Richige Experten | Richige Experten |
| inv_kl_div | Richige Experten | Richtige Experten | Richige Experten |

Es werden immer die Experten für die Klassifikation ausgewählt, welche den Task auch gelernt haben. Hier ist also alles richtig.  
vtab: ypsvqn17 (5 Experten) wird jedoch immer der falsche ausgewählt. Ab 3 Experten wird immer der dritte für Selection und Classification ausgewählt.  
vtab: p5604ebp selection around => gleichmäßige Verteilung der T. => Expert 2 übernimmt wieder gesammte Classification. Daten von T=2 scheinen sehr aus zu sehen wie alle anderen?


#### Zusätzlich:
- Wenn ein Experte immer wirder benutzt wird passt er sich evtl immer mehr dem Datensatz an und wird deswegen immer wahrschleinlicher für die Classification genutzt, aber ansich ist das kein großes Problem, da meisdoch der richtige Experte ausgewählt wird


## Felhler in Classification Vorgehen:
Fragen:  
- Warum werden auch Tasks schlechter, obwohl der Experte garnicht angefasst wurde?  
Wird der Head vernünftig geswitcht? Funktioniert switch_expert wirklich wie es soll?  
Testlauf 1 (klein): 2 Experten und dann kleiner testlauf. Testlauf 2 (groß): Echte Daten und Expertenmenge (evtl. overkill)
- Warum wird der Task beim gefinetunten Experten schlechter:  
switch_expert?  
- Zu starkes forgetting bei alten Tasks der gefintunten Experten?  
Die Acc. angucken
Run: uxywoac8 hat Expert 2 alle neues Tasks gelern:  
-> Tasks 0-3 bleiben dauerhaft gut. Aber auch reduce_Dataset

- Geziehlt immer gen gleichen Experten fintunen. bleiben die Tasks davor alle gut? anye6lip + andere: Fast, manchmal übernimmt der Experte, welcher mehr gefintuned wurde etwas von der Classification anderer.  
- Macht es einen unterschied ob die Funktionen aufgerufen werden oder nicht? Nein macht es nicht, zumindest nicht im trainloop: a3w17x17 vs v63d9w0z  

#### Ab jetzt tests mit num_exp = 2 und gesammten Dataset, weil reduce dataset die ergebnisse verfälscht, aber mit vtab, weil der Datensatz klein ist

#### tl6k8tw4: 2 Experten und des wird immer Expert 1 zu finetunen benutzt. Mal gucken was passiert  
#### mh9c20d2: 3 Experts:  
1. Erstes Finetuning (Task3): Acc.(T3) = 0.0. 
1. Task der am Anfang von Experten gelernt wurde behält gute Acc.  
1. Der Task der davor von anderem Experten gelernt wurde hat deutlich schlechtere Acc.  
Gleiches Verhalten mit Index = 2 ??
2. Ursprünglicher Task bleibt weiter gut.
2. Experte übernimmt 2/3 con Expert2's Klassifizierung  
2. Experte Experte übernimmt Klassifikation der folgenden tasks, aber mit Acc = oder fast = 0

3. Siehe 2.  

#### --> ft_epochs hochstellen = 5 (fht6u6sa)
1. Gleich, aber er schnappt jetzt schon Classification von Task 1 weg
2. Er übernimmt die Classification von allem.
2. Ursprünglicher Task geht auf 0    
-> ft_epochs hoch macht alles schlechter


#### mh9c20d2 aber mit index = 2
1. Experte klaut bisschen Task0 Classification, abder sonst alles gut

2. Siehe 1. + Ursprünglicher Task 40%Acc. weg

3. Siehe 1. + Ursprünglicher Task 10%Acc. weg

#### hp5tfy7l Höhere lr (0.02) und mehr ft_epochs (3), index = 2
Idee: Lr sehr hoch machen und nur einen expterten 1 mal finetunen und dann abbrechen und gucken ob nur der Experte schlechter wurde, oder ob noch andere angepasst wurden, exit after T=3.  
- Komisch: alles gut, Acc: 87%


benutzt meine GPU: 9lkoch hab ich versucht zu killen, ging nicht  
benutzt 3. GPU und evtl auch meine: 9hlehman

#### Besprechung 01.04.
- Ich weiß jetzt das Kriterium (Average). Wenn man Max nimmt und dann inverse, erhält man wahrscheinlich die beste Verteilung der Tasks auf die Experten.
- Cifar100: Tests wurde für Classification immer der richtige Experte Ausgewählt. Die Klassifikation ist dann aber trotzdem (sehr) schlecht.
- Vtab: Bei 2 Experten mit around selection: Hä? (nw7i5zmf)
1. Auswahl gut, Acc. T1/T2 = 0, aber Expert1(T1) wurde garnicht angefasst
2. Auswahl schlecht, Acc. T1/T2 = 0, aber T3 sehr gut 
3. Auswahl schlechter, Acc. T1/3 = 0,T2 schlecht, T4 ca. random
4. Auswahl schlechter, Acc. T1/3 = 0,T2 schlecht, T4 ca. random, T5 sehr gut
5. Auswahl schlechter, Acc. T1/3/4/6 = 0, T2/5 schlecht
6. Siehe 5. + T7 =65%
7. Alles = 0/schlecht außer T0  
-> Acc. T0 verhält sich gut, nimmt stetig ab, aber wie zu erwarten  
-> Auswahl bei Classification verschiebt sich immer mehr Richtung Expert1
- Vtab: Bei 3 Experten mit around selection: Expert 2 übernimmt gesammte Classification (p5604ebp)  
Ab 3 Experten immer Expert2 für CLassification ausgewählt (für jeden Task des Runs), egal wer den Task gelernt hat und dann ist die Acc. immer ca. 0.
0. Training Expert2: Übernahme gesammter Classification für alle weiteren Tasks
1. Acc. T3 ist 44%, obwohl er von E0 gelernt wurde.
2. Acc. T4 ist 36%, obwohl er von E1 gelernt wurde.
3. Acc. T5 ist 40%, obwohl er von E2 gelernt wurde.
3. Acc. T5 ist 40%, obwohl er von E3 gelernt wurde.
##### -----
- seed rausnehmen
- Numerische Probleme (over/underflow)?
- Acc. von 0 ist sehr komisch, schlechter als Random geht eigentlich nicht
- Was sind die konkreten Zahlen?
1. Classification: Auswahl + Eigentliche Werte der Predictions (cifar)
2. Training: Abstandsmaße, warum immer wieder der gleicher E (vtab)

#### --moe_max_experts 2 --dataset vtab --lr 0.02 --finetune_epochs 3 --selection around --exit_after_T 2 --T 25 --batch_size 64
2 Experten, 3 Tasks.  
T2 wird richtiger E0 ausgewählt.
T1/2 Acc. = 0, warum, das geht eigentlich nicht.  
### hab den ersten Fehler!!!  
Testing: 13nxga69 hat nur bei cifar 60% geholt.  
4sjawej6 E2 übernimmt immernoch alle Classifications. 
#### These: In T2 ist eine Klasse die allen anderen sehr ähnlich sieht, deswegen wird immer nur E2 ausgewählt.
- Wenn --T=25 tritt müsste bei Task 5-8 auftreten (Klasse in Normalem T3 ist anscheinend zu allem am ähnlichsten. Das ist eine der Klassen 11-15)  
--> Das problem tritt nicht auf
- T:50, E:50: dulcet-snowflake-86 Sehr schlecht
- T:50, E:50, exit_after_T:1: Classification von T1 von E0 übernommen
1. Generell sind die Log_probs verdächtig niedrig.   
Idee: Fehlerquelle: Bei Erstellung der distr. werden die features geflipped (wahrscheinlich für robisteres Modell), habe das jetzt auskommentiert und jetzt ist die Classification besser. Aber eigentlich dürfte das nicht der Fehler sein. ist es auch nicht!  
--> Es gibt zum ersten mal eine positive/realistische Prob. mit E3, er übernimmt fälschlicherweise gesammte Classification.  
Idee: Fehlerquelle: bei Erstellen der Distr. Test: Erstellen und gleiche Daten durchjagen, gucken ob die Werte hoch sind.  
--> Ja sind sie, alle bei ca. 1000.  
Idee: wo geht's dazwischen verloren?
